In [9]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation , Embedding , Flatten, Dropout, Convolution1D
from keras.callbacks import Callback
from keras.preprocessing import sequence
from keras.optimizers import Adam
import re

#from keras.callbacks import TensorBoard
# males -> 1
# females -> 0

#read processed data
df_processed = pd.read_json("out.json")
print("df head: " + str(df_processed.head()))

values_processed = df_processed.values
docs_padded_string = values_processed[:,2]
# calculate max_length based on the first entry of the 'post' column's encoded string (it conteins the encoded ints
# separated by ','-s so it will have to be split and converted back to int before use.)
max_length = len(docs_padded_string[0].split(','))
print("max_length: " + str(max_length))
padded_docs = np.zeros((len(docs_padded_string),max_length))
# convert the string to a list of ints by splitting them along ',' characters:
for i in range(len(docs_padded_string)):
    padded_docs[i] = [int(x) for x in docs_padded_string[i].split(",")]
    
#print("padded docs string: " + str(docs_padded_string[0]))
#print("padded docs: " + str(padded_docs[0]))
gender_list = values_processed[: , 1]
# check finalized word_dict:


# get valid/test/train split
valid_split = 0.2
test_split = 0.1
nb_samples = values_processed.shape[0]




X_train = padded_docs[0:int(nb_samples*(1-valid_split-test_split))]
Y_train = gender_list[0:int(nb_samples*(1-valid_split-test_split))]
X_valid = padded_docs[int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
Y_valid = gender_list[int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
X_test  = padded_docs[int(nb_samples*(1-test_split)):]
Y_test  = gender_list[int(nb_samples*(1-test_split)):]

# check shapes
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_valid shape: " + str(X_valid.shape))
print("Y_valid shape: " + str(Y_valid.shape))# define the model
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_length))

# Convolutional model (3x conv, flatten, 2x dense)
model.add(Convolution1D(64, 5, padding='same'))
model.add(Convolution1D(32, 5, padding='same'))
model.add(Convolution1D(16, 5, padding='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

model.fit(X_train, Y_train, epochs=3, batch_size=64)

# Evaluation on the test set
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


#todo:
#sgd momentum nastarov learning-rate



df head:       age  gender                                               post
0      27       1  5050,1251,5657,1154,6286,2236,2376,147,6129,68...
1      25       1  9779,7676,8862,8989,1659,9790,5615,8613,586,63...
10     37       0  8842,890,890,890,890,5547,4348,9676,3981,130,2...
100    27       1  5139,7939,7939,7058,8956,6853,4203,2796,4140,8...
1000   26       1  628,9876,8810,2376,6928,8810,6742,7915,3104,24...
max_length: 500
X_train shape: (7000, 500)
Y_train shape: (7000,)
X_valid shape: (2000, 500)
Y_valid shape: (2000,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 500, 64)           10304     
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 500, 32)           10272     